# Solution

## Data loading

## Feature extraction

## Learning

In [ ]:
X_fit, X_val, y_fit, y_val = model_selection.train_test_split(
    features, targets,
    random_state=42
)

In [ ]:
fit = lgb.Dataset(X_fit, y_fit)
val = lgb.Dataset(X_val, y_val, reference=fit)

model = lgb.train(
    params={
        'learning_rate': 0.1,
        'objective': 'binary',
        'metric': 'auc',
        'boost_from_average': True,
        'min_data_in_leaf': 20,
        'max_bin': 255,
        'num_leaves': 31,
        'min_data_per_group': 100,
        'cat_smooth': 10,
        'seed': 42
    },
    train_set=fit,
    num_boost_round=10_000,
    valid_sets=(fit, val),
    valid_names=('fit', 'val'),
    early_stopping_rounds=20,
    verbose_eval=100
)

In [ ]:
importances = model.feature_importance(importance_type='gain').astype(int)
print(pd.Series(importances, index=X_fit.columns).sort_values(ascending=False))

In [ ]:
val_score = metrics.roc_auc_score(y_val, model.predict(X_val))
model.save_model(f'models/model_{val_score:.4f}.lgb')